In [1]:
import os
import sys
from time import time, sleep
import json
from pathlib import Path
import datetime
import collections

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
scraped_data_directories = ["../data/kifs/001d"]
scraped_data_directories = [Path(d) for d in scraped_data_directories]

In [ ]:
len()

In [7]:
files = [file for directory in scraped_data_directories for file in directory.iterdir() if file.suffix == ".json" and file.stem != "__output__"]
files.sort()
len(files)

500

In [ ]:
def dump_kif(kif_json, kif_info_json, f=None):
    assert kif_json["info"]["EpisodeId"] == kif_info_json["id"]
    print("kore_kif_v1", file=f)
    print(f'EpisodeId{kif_json["info"]["EpisodeId"]}', file=f)
    print(f'submissionId{kif_info_json["agents"][0]["submissionId"]}', file=f)
    print(f'submissionId{kif_info_json["agents"][1]["submissionId"]}', file=f)
    
    for i, step in enumerate(kif_json["steps"]):
        if i == 0:
            print("===", file=f)
        else:
            print("---", file=f)
        for player in range(2):
            player_actions = step[player]["action"]
            print(len(player_actions), file=f)
            for shipyard_id, action in player_actions.items():
                print(shipyard_id, action, file=f)
        
        print(i, file=f)
        kores = step[0]["observation"]["kore"]
        for y in range(21):
            row = [int(k) if k == int(k) else k for k in kores[y*21:y*21+21]]
            print(*row, file=f)
        
        for player in range(2):
            print(step[player]["observation"]["remainingOverageTime"], file=f)
            player_kore, shipyards, fleets = step[0]["observation"]["players"][player]
            print(player_kore, file=f)
            print(len(shipyards), file=f)
            for shipyard_id, shipyard in shipyards.items():
                assert len(shipyard) == 3
                assert int(shipyard_id.split("-")[1]) <= 400
                assert int(shipyard_id.split("-")[1]) < 100
                print(shipyard_id, *shipyard, file=f)
            print(len(fleets), file=f)
            for fleet_id, fleet in fleets.items():
                assert len(fleet) == 5
                assert int(fleet_id.split("-")[1]) <= 400
                assert int(fleet_id.split("-")[1]) < 100
                if fleet[4] == "":
                    fleet = fleet[:4] + ["null"]
                print(fleet_id, *fleet, file=f)
    print("===", file=f)
    print(-1, file=f)
    print(*kif_json["rewards"], file=f)

#dump_kif(kif_json, kif_info_json)

In [ ]:
for kif_file, kif_info_file in zip(files[::2], files[1::2]):
    with open(kif_file) as f:
        kif_json = json.load(f)
    with open(kif_info_file) as f:
        kif_info_json = json.load(f)
    with open(f"./{kif_file.stem}.kif", "w") as f:
        dump_kif(kif_json, kif_info_json, f)